In [6]:
import pandas as pd
import openpyxl # type: ignore
import unicodedata

Funciones

In [4]:
# Función para limpiar valores
def limpiar_valores(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', errors='ignore').decode('utf-8')
        value = value.strip()
    elif pd.isnull(value):
        # Sustituir null por un espacio en blanco
        value = ''
    return value

# Función para clasificar en quinquenios
def clasificar(edad):
    if pd.isna(edad):
        return "desconocido"
    inicio = (edad // 5) * 5
    fin = inicio + 4
    return f'{inicio}-{fin}'

In [7]:
# Cargar el archivo Excel y limpiar valores
df = pd.read_excel("E://files_pandas//dengue//DENGUE141024.xlsx")
df = df.applymap(limpiar_valores)

# Filtrar valores específicos
df = df[df["DES_INS_UNIDAD"].str.strip().isin(["IMSS_OPD", "SSA"])]


C:\Users\najer\AppData\Local\Temp\ipykernel_16220\859555970.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_valores)
